## Instructions ##
1. Map top 10% snps to genes
2. Use additive model 


In [351]:
import pandas as pd 
import numpy as np
import bisect as bs
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import scipy.stats as stat
import math

In [2]:
SNP_url = "./1011geneSNP.csv"
Gene_url = "./GCF_000146045.2_R64_genomic.gff"

snp_yeast_matrix = pd.read_csv(SNP_url, sep = ',' , error_bad_lines=False)
del snp_yeast_matrix['#CHROM']
pos_list = list(snp_yeast_matrix['POS'])
del snp_yeast_matrix['POS']
del snp_yeast_matrix['REF']
del snp_yeast_matrix['ALT']
del snp_yeast_matrix['ANN[*].GENE']
del snp_yeast_matrix['ANN[*].GENEID']
index_list = snp_yeast_matrix["ID"].tolist()
del snp_yeast_matrix['ID']
snp_yeast_matrix = snp_yeast_matrix.transpose()
column_names = list(snp_yeast_matrix.index)
to_delete = ["ABC", "ABF", "ASP", "BGS","BGF", "BHL","BHQ","BID","BIR","BKG","BKN","CFH","SACE_YAL","SACE_YBA",
             "SACE_YBB","SACE_YBM","SACE_YBN","SACE_YBO","SACE_YBP","SACE_YCJ","SACE_YCS","SACE_YCT","SACE_YCU","SACE_YCV","SACE_YCW",
             "SACE_YCX","SACE_YCY","SACE_YCZ","SACE_YDA","SACE_YDB","SACE_YDC","SACE_YDD", "SACE_YDE","SACE_YDF","SACE_YDG","SACE_YDH","SACE_YDI",
             "SACE_YDJ", "SACE_YDK", "SACE_YDL"]
print(len(to_delete))
index = 0
for index in range(len(to_delete)):
    curr = to_delete[index]
    curr = curr+"_"+curr
    to_delete[index] = curr
    index+=1
to_delete_index = []
for item in to_delete:
    curr_index = column_names.index(item)
    to_delete_index.append(curr_index)
snp_yeast_matrix = snp_yeast_matrix.drop(to_delete)
snp_yeast_matrix.head()

40


,0,1,2,3,4,5,6,7,8,9,...,18921,18922,18923,18924,18925,18926,18927,18928,18929,18930
AAA_AAA,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAB_AAB,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAC_AAC,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAD_AAD,0,0,0,0,1,1,1,0,0,0,...,0,0,1,1,1,1,0,0,1,1
AAE_AAE,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]
total_run = 100

## Calculate index of top associated snps in 100 runs ##

In [4]:
sum = 0
weight_vector_1 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPD6AU, test_size=0.2)
    lr1 = linear_model.LinearRegression()
    lr1.fit(x_train, y_train)
    temp = np.array(lr1.coef_)
    weight_vector_1 = [x + y for x, y in zip(weight_vector_1, temp)]
    predictions_1 = lr1.predict(x_test)
    pcc_1 = stat.pearsonr(predictions_1, y_test)
    counter+=1
    sum += pcc_1[0]

avg_assoc_1 = sum/total_run
avg_weight_vector_1 = np.divide(weight_vector_1, total_run)
print(avg_assoc_1)
print(avg_weight_vector_1)
top2000_index = (-avg_weight_vector_1).argsort()[:2000]
print (top2000_index)
print (avg_weight_vector_1[top2000_index])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_2 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDBENOMYL200, test_size=0.2)
    lr2 = linear_model.LinearRegression()
    lr2.fit(x_train, y_train)
    temp = np.array(lr2.coef_)
    weight_vector_2 = [x + y for x, y in zip(weight_vector_2, temp)]
    predictions_2 = lr2.predict(x_test)
    pcc_2 = stat.pearsonr(predictions_2, y_test)
    counter+=1
    sum += pcc_2[0]

avg_assoc_2 = sum/total_run
avg_weight_vector_2 = np.divide(weight_vector_2, total_run)
print(avg_assoc_2)
print(avg_weight_vector_2)
top2000_index_2 = (-avg_weight_vector_2).argsort()[:2000]
print (top2000_index_2)
print (avg_weight_vector_2[top2000_index_2])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_3 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDNACL1M, test_size=0.2)
    lr3 = linear_model.LinearRegression()
    lr3.fit(x_train, y_train)
    temp = np.array(lr3.coef_)
    weight_vector_3 = [x + y for x, y in zip(weight_vector_3, temp)]
    predictions_3 = lr3.predict(x_test)
    pcc_3 = stat.pearsonr(predictions_3, y_test)
    counter+=1
    sum += pcc_3[0]

avg_assoc_3 = sum/total_run
avg_weight_vector_3 = np.divide(weight_vector_3, total_run)
print(avg_assoc_3)
print(avg_weight_vector_3)
top2000_index_3 = (-avg_weight_vector_3).argsort()[:2000]
print (top2000_index_3)
print (avg_weight_vector_3[top2000_index_3])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_4 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDSDS, test_size=0.2)
    lr4 = linear_model.LinearRegression()
    lr4.fit(x_train, y_train)
    temp = np.array(lr4.coef_)
    weight_vector_4 = [x + y for x, y in zip(weight_vector_4, temp)]
    predictions_4 = lr4.predict(x_test)
    pcc_4 = stat.pearsonr(predictions_4, y_test)
    counter+=1
    sum += pcc_4[0]

avg_assoc_4 = sum/total_run
avg_weight_vector_4 = np.divide(weight_vector_4, total_run)
print(avg_assoc_4)
print(avg_weight_vector_4)
top2000_index_4 = (-avg_weight_vector_4).argsort()[:2000]
print (top2000_index_4)
print (avg_weight_vector_4[top2000_index_4])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_5 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGALACTOSE, test_size=0.2)
    lr5 = linear_model.LinearRegression()
    lr5.fit(x_train, y_train)
    temp = np.array(lr5.coef_)
    weight_vector_5 = [x + y for x, y in zip(weight_vector_5, temp)]
    predictions_5 = lr5.predict(x_test)
    pcc_5 = stat.pearsonr(predictions_5, y_test)
    counter+=1
    sum += pcc_5[0]

avg_assoc_5 = sum/total_run
avg_weight_vector_5 = np.divide(weight_vector_5, total_run)
print(avg_assoc_5)
print(avg_weight_vector_5)
top2000_index_5 = (-avg_weight_vector_5).argsort()[:2000]
print (top2000_index_5)
print (avg_weight_vector_5[top2000_index_5])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_6 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGLYCEROL, test_size=0.2)
    lr6 = linear_model.LinearRegression()
    lr6.fit(x_train, y_train)
    temp = np.array(lr6.coef_)
    weight_vector_6 = [x + y for x, y in zip(weight_vector_6, temp)]
    predictions_6 = lr6.predict(x_test)
    pcc_6 = stat.pearsonr(predictions_6, y_test)
    counter+=1
    sum += pcc_6[0]

avg_assoc_6 = sum/total_run
avg_weight_vector_6 = np.divide(weight_vector_6, total_run)
print(avg_assoc_6)
print(avg_weight_vector_6)
top2000_index_6 = (-avg_weight_vector_6).argsort()[:2000]
print (top2000_index_6)
print (avg_weight_vector_6[top2000_index_6])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_7 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPRIBOSE, test_size=0.2)
    lr7 = linear_model.LinearRegression()
    lr7.fit(x_train, y_train)
    temp = np.array(lr7.coef_)
    weight_vector_7 = [x + y for x, y in zip(weight_vector_7, temp)]
    predictions_7 = lr7.predict(x_test)
    pcc_7 = stat.pearsonr(predictions_7, y_test)
    counter+=1
    sum += pcc_7[0]

avg_assoc_7 = sum/total_run
avg_weight_vector_7 = np.divide(weight_vector_7, total_run)
print(avg_assoc_7)
print(avg_weight_vector_7)
top2000_index_7 = (-avg_weight_vector_7).argsort()[:2000]
print (top2000_index_7)
print (avg_weight_vector_7[top2000_index_7])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_8 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPSORBITOL, test_size=0.2)
    lr8 = linear_model.LinearRegression()
    lr8.fit(x_train, y_train)
    temp = np.array(lr8.coef_)
    weight_vector_8 = [x + y for x, y in zip(weight_vector_8, temp)]
    predictions_8 = lr8.predict(x_test)
    pcc_8 = stat.pearsonr(predictions_8, y_test)
    counter+=1
    sum += pcc_8[0]

avg_assoc_8 = sum/total_run
avg_weight_vector_8 = np.divide(weight_vector_8, total_run)
print(avg_assoc_8)
print(avg_weight_vector_8)
top2000_index_8 = (-avg_weight_vector_8).argsort()[:2000]
print (top2000_index_8)
print (avg_weight_vector_8[top2000_index_8])

0.012257347815486095
[-4.70382797e+07 -6.89749386e+07 -1.83195975e+07 ...  2.27410609e+05
  1.05730682e+04 -2.23611876e+06]
[    3    21    10 ... 14519 14741  1267]
[63489555.34764801 37168502.67938235 25187533.21800183 ...
   966291.92206568   966192.06802127   966080.69411545]
0.02928149642082002
[11362998.05945773 28713208.33432954 13627430.90108617 ...
   -40442.2075441    165859.89389064   -39600.91799881]
[   1    3    7 ... 5896 1129 4482]
[28713208.33432954 23918107.46240122 22478385.36228473 ...
   322581.45664118   322579.76917957   322544.18034099]
0.03055584375431484
[-1.30255456e+07  1.16265475e+07  7.70332847e+07 ...  6.63462302e+04
  2.67797084e+05  1.03873146e+06]
[    2     5    11 ... 13456  9317 17605]
[77033284.7110372  34113975.27897751 28712000.58812311 ...
   985321.52233595   985242.63674778   984991.83188687]
0.02986436825179075
[-31589016.07814947 -32537335.13906072 -32012055.36644748 ...
    -50114.80156448   -819854.95537944   -350007.71854033]
[    4    37

In [5]:
valid_snp_list=[top2000_index,top2000_index_2,top2000_index_3,top2000_index_4,top2000_index_5,top2000_index_6,top2000_index_7,top2000_index_8]

In [454]:
def get_gene(input):
    words = input.split(';')
    gene_tuple = words[2]
    words = gene_tuple.split('=')
    gene = words[1]
    return gene

def snp_pos_to_gene(pos):
    index = bs.bisect(start_pos, pos)
    curr_start = start_pos[index-1]
    curr_tuple = dict[curr_start]
    curr_end = curr_tuple[2]
    if pos in range ((int)(curr_start), (int)(curr_end)):
        return curr_tuple[0]

names=["Contig", "Seq","Type", "Start", "End", "N/A", "N/A1", "N/A2", "Info"]
gene_matrix = pd.read_csv(Gene_url, sep = '\t', error_bad_lines=False, skiprows = 8, names = names)
gene_matrix.head()

dict = {}
gene_list = []
for row in gene_matrix.itertuples():
    if(row.Type == "gene"):
        curr_info = row.Info
        curr_gene = get_gene(curr_info)
        gene_list.append(curr_gene)
        curr_start = row.Start
        curr_end = row.End
        tuple = (curr_gene, curr_start, curr_end)
        dict[curr_start] = tuple
print("Dict finished. There are %u genes" %len(dict))
start_pos = list(dict.keys())
rows = snp_yeast_matrix.index.values
rows = rows[0:]
array = np.zeros([len(rows),len(gene_list)])

yeast_gene_matrix_1 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_2 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_3 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_4 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_5 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_6 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_7 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_8 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)

yeast_gene_matrix_list = [yeast_gene_matrix_1,yeast_gene_matrix_2,yeast_gene_matrix_3,yeast_gene_matrix_4,yeast_gene_matrix_5,yeast_gene_matrix_6,yeast_gene_matrix_7,yeast_gene_matrix_8]
yeast_gene_matrix_1.head()

Dict finished. There are 6376 genes


,PAU8,YAL067W-A,SEO1,YAL065C,YAL064W-B,TDA8,YAL064W,YAL063C-A,FLO9,GDH3,...,tN(GUU)Q,tM(CAU)Q1,COX2,Q0255,tF(GAA)Q,tT(UAG)Q2,tV(UAC)Q,COX3,tM(CAU)Q2,RPM1
AAA_AAA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAB_AAB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAC_AAC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAD_AAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAE_AAE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [474]:
print(gene_matrix["Seq"])

0        RefSeq
1        RefSeq
2        RefSeq
3        RefSeq
4        RefSeq
5        RefSeq
6        RefSeq
7        RefSeq
8        RefSeq
9        RefSeq
10       RefSeq
11       RefSeq
12       RefSeq
13       RefSeq
14       RefSeq
15       RefSeq
16       RefSeq
17       RefSeq
18       RefSeq
19       RefSeq
20       RefSeq
21       RefSeq
22       RefSeq
23       RefSeq
24       RefSeq
25       RefSeq
26       RefSeq
27       RefSeq
28       RefSeq
29       RefSeq
          ...  
26915    RefSeq
26916    RefSeq
26917    RefSeq
26918    RefSeq
26919    RefSeq
26920    RefSeq
26921    RefSeq
26922    RefSeq
26923    RefSeq
26924    RefSeq
26925    RefSeq
26926    RefSeq
26927    RefSeq
26928    RefSeq
26929    RefSeq
26930    RefSeq
26931    RefSeq
26932    RefSeq
26933    RefSeq
26934    RefSeq
26935    RefSeq
26936    RefSeq
26937    RefSeq
26938    RefSeq
26939    RefSeq
26940    RefSeq
26941    RefSeq
26942    RefSeq
26943    RefSeq
26944       NaN
Name: Seq, Length: 26945

In [455]:
def snp_list_to_gene(valid_snp_list, snp_yeast_matrix, yeast_gene_matrix):
    max_snp = 18931
    max_yeast = 971
    snp_counter = 0
    for snp_counter in range(max_snp):
        if(snp_counter %2000 == 0):
            print(snp_counter)
        curr_column = snp_yeast_matrix[snp_counter].tolist()
        curr_pos = pos_list[snp_counter]
        yeast_counter = 0
        for yeast_counter in range (max_yeast):
            curr_val = curr_column[yeast_counter]
            if(curr_val)!=0:
                if snp_counter in valid_snp_list:
                    gene = snp_pos_to_gene(curr_pos)
                    if(gene is None):
                        continue
                    yeast_name = rows[yeast_counter]
                    yeast_gene_matrix[gene][yeast_name] += 1
            yeast_counter += 1
        snp_counter += 1
    print("done")


In [456]:
for valid_snp, yg_matrix in zip(valid_snp_list, yeast_gene_matrix_list):
    snp_list_to_gene(valid_snp, snp_yeast_matrix, yg_matrix)


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done


In [457]:
min_yeast_with_gene = 5
min_total_gene = 10
for matrix in yeast_gene_matrix_list:
    for column in matrix:
        counter = 0
        value = 0
        temp = matrix[column]
        for element in temp:
            if element != 0:
                counter += 1
                value += 4
        if(counter < min_yeast_with_gene and value <min_total_gene ):
            del matrix[column]
            
for matrix in yeast_gene_matrix_list:
    print(matrix.shape)


(971, 376)
(971, 371)
(971, 377)
(971, 369)
(971, 370)
(971, 361)
(971, 369)
(971, 368)


In [458]:
yeast_gene_matrix_list[0]
 

,YAL064W,YAL063C-A,FLO9,GDH3,BDH2,BDH1,CNE1,GPB2,PEX22,ACS1,...,HDA3,AOS1,SEC23,DPM1,GDB1,ATG13,PZF1,SKI3,RPC82,QCR2
AAA_AAA,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AAB_AAB,0.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,1.0,3.0,...,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
AAC_AAC,1.0,0.0,4.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,...,2.0,1.0,2.0,0.0,0.0,2.0,0.0,1.0,3.0,1.0
AAD_AAD,1.0,1.0,8.0,1.0,1.0,4.0,6.0,6.0,1.0,8.0,...,2.0,2.0,4.0,1.0,5.0,5.0,0.0,3.0,4.0,1.0
AAE_AAE,0.0,0.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,2.0,...,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
AAG_AAG,0.0,1.0,10.0,1.0,0.0,3.0,3.0,2.0,1.0,6.0,...,1.0,2.0,2.0,1.0,2.0,4.0,0.0,2.0,0.0,1.0
AAH_AAH,1.0,1.0,8.0,1.0,1.0,2.0,4.0,5.0,1.0,6.0,...,2.0,1.0,0.0,0.0,3.0,3.0,0.0,2.0,2.0,1.0
AAI_AAI,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,...,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
AAK_AAK,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,...,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
AAL_AAL,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,0.0,2.0,2.0,1.0,2.0,2.0,1.0


In [113]:
yeast_gene_matrix_list[1].head()


,FLO9,BDH2,BDH1,ECM1,CNE1,GPB2,ACS1,FLC2,OAF1,AIM2,...,TIF3,MMS1,BSP1,VPS4,DPB2,BET2,AOS1,GDB1,ATG13,PZF1
AAA_AAA,4.0,1.0,1.0,2.0,1.0,0.0,3.0,1.0,2.0,2.0,...,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0
AAB_AAB,0.0,1.0,0.0,1.0,3.0,1.0,3.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
AAC_AAC,6.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0,...,1.0,2.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0
AAD_AAD,11.0,2.0,1.0,1.0,5.0,6.0,5.0,4.0,6.0,1.0,...,1.0,3.0,3.0,2.0,0.0,0.0,4.0,8.0,2.0,3.0
AAE_AAE,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,2.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


## Neural Network ##

In [459]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as dt

class Net(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size, lr):
        super(Net, self).__init__()
        self.l1 = nn.Linear(input_size, h1)
        self.l2 = nn.Linear(h1, h2)
        self.l3 = nn.Linear(h2, h3)
        self.l4 = nn.Linear(h3, output_size)
        self.relu = nn.ReLU()
        
        self.loss_fn = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr)

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        return out
     
    def train_net(self, num_epochs, train_loader, test_loader):
        max_rsq = 0
        max_epoch = 0
        for epoch in range (num_epochs):
            total_loss = 0
            counter = 0
            for i, (data, target) in enumerate(train_loader):
                self.zero_grad()
                data = data.to(device)
                target = target.to(device)
                output = self.forward(data).to(device)
                loss = self.loss_fn(output, target)
                total_loss += loss
                counter += 1
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            
            # empty arrays for empty tensor 
            empty1 = np.empty((0,1))
            empty2 = np.empty((0,1))
            output_tensor = torch.tensor(empty1, dtype = torch.float)
            target_tensor = torch.tensor(empty2, dtype = torch.float)
            # concatenate output and target to the empty tensors 
            for i, (data, target) in enumerate(test_loader): 
                data = data.to(device) 
                target = target.to(device) 
                output = self.forward(data).to(device)
                target = torch.reshape(target, (len(target),1))
                output_tensor = torch.cat((output_tensor, output.cpu()))
                target_tensor = torch.cat((target_tensor, target.cpu()))
            
            #convert the tensor into numpy arrays and calculate correlation 
            output_array = output_tensor.detach().numpy()
            target_array = target_tensor.detach().numpy()
            if ((output_array.shape[0]!=target_array.shape[0]) or (output_array.shape[1]!=target_array.shape[1])):
                raise Exception('Dimension mismatch')
            result = stat.pearsonr(target_array, output_array)
            print("Correlation: %f \n P-Value: %f" %(result[0], result[1])) 
            if(math.fabs(result[0])>math.fabs(max_rsq)):
                max_rsq = result[0]
                max_epoch = epoch
            epoch += 1
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                .format(epoch, num_epochs, total_loss/len(train_loader)))
    
        print("Max correlation is: {} at {}th epoch".format(max_rsq, max_epoch))

In [460]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]

In [461]:
device = torch.device('cuda')
h1 = 100
h2 = 50
h3 = 20
output_size = 1
lr = 0.001
num_epochs = 100
batch_size = 200
split = 0.8

In [462]:
training_data_0 = torch.tensor(yeast_gene_matrix_list[0].values, dtype = torch.float)
answer_0 = torch.tensor(YPD6AU, dtype = torch.float)
train_set_0 = dt.TensorDataset(training_data_0, answer_0)
train_size = int(split * len(train_set_0))
test_size = len(train_set_0) - train_size 
train_dataset_0, test_dataset_0 = dt.random_split(train_set_0, [train_size, test_size])
train_loader_0 = dt.DataLoader(dataset=train_dataset_0, batch_size=batch_size)
test_loader_0 = dt.DataLoader(dataset=test_dataset_0, batch_size=batch_size)


training_data_1 = torch.tensor(yeast_gene_matrix_list[1].values, dtype = torch.float)
answer_1 = torch.tensor(YPDBENOMYL200, dtype = torch.float)
train_set_1 = dt.TensorDataset(training_data_1, answer_1)
train_dataset_1, test_dataset_1 = dt.random_split(train_set_1, [train_size, test_size])
train_loader_1 = dt.DataLoader(dataset=train_dataset_1, batch_size=batch_size)
test_loader_1 = dt.DataLoader(dataset=test_dataset_1, batch_size=batch_size)


training_data_2 = torch.tensor(yeast_gene_matrix_list[2].values, dtype = torch.float)
answer_2 = torch.tensor(YPDNACL1M, dtype = torch.float)
train_set_2 = dt.TensorDataset(training_data_2, answer_2)
train_dataset_2, test_dataset_2 = dt.random_split(train_set_2, [train_size, test_size])
train_loader_2 = dt.DataLoader(dataset=train_dataset_2, batch_size=batch_size)
test_loader_2 = dt.DataLoader(dataset=test_dataset_2, batch_size=batch_size)


training_data_3 = torch.tensor(yeast_gene_matrix_list[3].values, dtype = torch.float)
answer_3 = torch.tensor(YPDSDS, dtype = torch.float)
train_set_3 = dt.TensorDataset(training_data_3, answer_3)
train_dataset_3, test_dataset_3 = dt.random_split(train_set_3, [train_size, test_size])
train_loader_3 = dt.DataLoader(dataset=train_dataset_3, batch_size=batch_size)
test_loader_3 = dt.DataLoader(dataset=test_dataset_3, batch_size=batch_size)

training_data_5 = torch.tensor(yeast_gene_matrix_list[4].values, dtype = torch.float)
answer_5 = torch.tensor(YPGALACTOSE, dtype = torch.float)
train_set_5 = dt.TensorDataset(training_data_5, answer_5)
train_dataset_5, test_dataset_5 = dt.random_split(train_set_5, [train_size, test_size])
train_loader_5 = dt.DataLoader(dataset=train_dataset_5, batch_size=batch_size)
test_loader_5 = dt.DataLoader(dataset=test_dataset_5, batch_size=batch_size)

training_data_6 = torch.tensor(yeast_gene_matrix_list[5].values, dtype = torch.float)
answer_6 = torch.tensor(YPGLYCEROL, dtype = torch.float)
train_set_6 = dt.TensorDataset(training_data_6, answer_6)
train_dataset_6, test_dataset_6 = dt.random_split(train_set_6, [train_size, test_size])
train_loader_6 = dt.DataLoader(dataset=train_dataset_6, batch_size=batch_size)
test_loader_6 = dt.DataLoader(dataset=test_dataset_6, batch_size=batch_size)

training_data_7 = torch.tensor(yeast_gene_matrix_list[6].values, dtype = torch.float)
answer_7 = torch.tensor(YPRIBOSE, dtype = torch.float)
train_set_7 = dt.TensorDataset(training_data_7, answer_7)
train_dataset_7, test_dataset_7 = dt.random_split(train_set_7, [train_size, test_size])
train_loader_7 = dt.DataLoader(dataset=train_dataset_7, batch_size=batch_size)
test_loader_7 = dt.DataLoader(dataset=test_dataset_7, batch_size=batch_size)

training_data_8 = torch.tensor(yeast_gene_matrix_list[7].values, dtype = torch.float)
answer_8 = torch.tensor(YPSORBITOL, dtype = torch.float)
train_set_8 = dt.TensorDataset(training_data_8, answer_8)
train_dataset_8, test_dataset_8 = dt.random_split(train_set_8, [train_size, test_size])
train_loader_8 = dt.DataLoader(dataset=train_dataset_8, batch_size=batch_size)
test_loader_8 = dt.DataLoader(dataset=test_dataset_8, batch_size=batch_size)



In [463]:
train_loader_list = [train_loader_0, train_loader_1, train_loader_2, train_loader_3, train_loader_5, train_loader_6, train_loader_7, train_loader_8]
test_loader_list = [test_loader_0, test_loader_1, test_loader_2, test_loader_3, test_loader_5, test_loader_6, test_loader_7, test_loader_8]

In [464]:
print(int(split * len(train_set_0)))

776


In [465]:
input_size = 376
net_0 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_0.train_net(num_epochs, train_loader_list[0], test_loader_list[0])

Correlation: -0.103957 
 P-Value: 0.148103
Epoch [1/100], Loss: 0.1223
Correlation: 0.177342 
 P-Value: 0.013130
Epoch [2/100], Loss: 0.0489
Correlation: 0.216044 
 P-Value: 0.002418
Epoch [3/100], Loss: 0.0358
Correlation: 0.230327 
 P-Value: 0.001198
Epoch [4/100], Loss: 0.0356
Correlation: 0.067649 
 P-Value: 0.347387
Epoch [5/100], Loss: 0.0297
Correlation: 0.037864 
 P-Value: 0.599216
Epoch [6/100], Loss: 0.0268
Correlation: 0.030737 
 P-Value: 0.669705
Epoch [7/100], Loss: 0.0239
Correlation: -0.003450 
 P-Value: 0.961793
Epoch [8/100], Loss: 0.0219
Correlation: -0.081135 
 P-Value: 0.259505
Epoch [9/100], Loss: 0.0212
Correlation: -0.117717 
 P-Value: 0.101221
Epoch [10/100], Loss: 0.0208
Correlation: -0.087992 
 P-Value: 0.221245
Epoch [11/100], Loss: 0.0204
Correlation: -0.071595 
 P-Value: 0.319923
Epoch [12/100], Loss: 0.0200
Correlation: -0.074027 
 P-Value: 0.303717
Epoch [13/100], Loss: 0.0197
Correlation: -0.086717 
 P-Value: 0.228040
Epoch [14/100], Loss: 0.0194
Correla

In [466]:
input_size = 371
net_1 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_1.train_net(num_epochs, train_loader_list[1], test_loader_list[1])

Correlation: 0.000605 
 P-Value: 0.992849
Epoch [1/100], Loss: 0.0158
Correlation: 0.022077 
 P-Value: 0.759345
Epoch [2/100], Loss: 0.0111
Correlation: -0.001336 
 P-Value: 0.984953
Epoch [3/100], Loss: 0.0105
Correlation: -0.134255 
 P-Value: 0.061318
Epoch [4/100], Loss: 0.0100
Correlation: -0.078696 
 P-Value: 0.274143
Epoch [5/100], Loss: 0.0098
Correlation: 0.040840 
 P-Value: 0.570801
Epoch [6/100], Loss: 0.0097
Correlation: 0.009288 
 P-Value: 0.897473
Epoch [7/100], Loss: 0.0097
Correlation: 0.025219 
 P-Value: 0.726363
Epoch [8/100], Loss: 0.0096
Correlation: 0.043958 
 P-Value: 0.541735
Epoch [9/100], Loss: 0.0096
Correlation: 0.035506 
 P-Value: 0.622164
Epoch [10/100], Loss: 0.0096
Correlation: 0.001756 
 P-Value: 0.980513
Epoch [11/100], Loss: 0.0096
Correlation: -0.011915 
 P-Value: 0.868687
Epoch [12/100], Loss: 0.0096
Correlation: -0.002773 
 P-Value: 0.969311
Epoch [13/100], Loss: 0.0095
Correlation: 0.002834 
 P-Value: 0.968606
Epoch [14/100], Loss: 0.0095
Correlatio

In [467]:
input_size = 377
net_2 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_2.train_net(num_epochs, train_loader_list[2], test_loader_list[2])

Correlation: -0.029119 
 P-Value: 0.686139
Epoch [1/100], Loss: 0.0341
Correlation: 0.059690 
 P-Value: 0.407160
Epoch [2/100], Loss: 0.0134
Correlation: 0.142289 
 P-Value: 0.047223
Epoch [3/100], Loss: 0.0118
Correlation: 0.023395 
 P-Value: 0.745450
Epoch [4/100], Loss: 0.0116
Correlation: 0.091103 
 P-Value: 0.205285
Epoch [5/100], Loss: 0.0115
Correlation: 0.054921 
 P-Value: 0.445710
Epoch [6/100], Loss: 0.0110
Correlation: 0.094024 
 P-Value: 0.191067
Epoch [7/100], Loss: 0.0110
Correlation: 0.049966 
 P-Value: 0.487880
Epoch [8/100], Loss: 0.0108
Correlation: 0.053566 
 P-Value: 0.457038
Epoch [9/100], Loss: 0.0108
Correlation: 0.036718 
 P-Value: 0.610324
Epoch [10/100], Loss: 0.0107
Correlation: 0.036258 
 P-Value: 0.614806
Epoch [11/100], Loss: 0.0107
Correlation: 0.048865 
 P-Value: 0.497533
Epoch [12/100], Loss: 0.0107
Correlation: 0.045162 
 P-Value: 0.530718
Epoch [13/100], Loss: 0.0107
Correlation: 0.041046 
 P-Value: 0.568859
Epoch [14/100], Loss: 0.0106
Correlation: 0

In [468]:
input_size = 369
net_3 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_3.train_net(num_epochs, train_loader_list[3], test_loader_list[3])

Correlation: 0.170849 
 P-Value: 0.016939
Epoch [1/100], Loss: 0.0693
Correlation: 0.119016 
 P-Value: 0.097482
Epoch [2/100], Loss: 0.0404
Correlation: -0.086658 
 P-Value: 0.228356
Epoch [3/100], Loss: 0.0327
Correlation: -0.068944 
 P-Value: 0.338213
Epoch [4/100], Loss: 0.0315
Correlation: -0.036250 
 P-Value: 0.614894
Epoch [5/100], Loss: 0.0306
Correlation: 0.071234 
 P-Value: 0.322375
Epoch [6/100], Loss: 0.0292
Correlation: 0.040348 
 P-Value: 0.575460
Epoch [7/100], Loss: 0.0285
Correlation: -0.059568 
 P-Value: 0.408116
Epoch [8/100], Loss: 0.0277
Correlation: -0.070849 
 P-Value: 0.324999
Epoch [9/100], Loss: 0.0272
Correlation: 0.011284 
 P-Value: 0.875566
Epoch [10/100], Loss: 0.0268
Correlation: -0.005060 
 P-Value: 0.944001
Epoch [11/100], Loss: 0.0266
Correlation: -0.103612 
 P-Value: 0.149459
Epoch [12/100], Loss: 0.0265
Correlation: -0.099167 
 P-Value: 0.167803
Epoch [13/100], Loss: 0.0264
Correlation: -0.060355 
 P-Value: 0.401945
Epoch [14/100], Loss: 0.0264
Correl

In [469]:
input_size = 370
net_4 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_4.train_net(num_epochs, train_loader_list[4], test_loader_list[4])

Correlation: 0.150088 
 P-Value: 0.036236
Epoch [1/100], Loss: 0.6821
Correlation: 0.135224 
 P-Value: 0.059453
Epoch [2/100], Loss: 0.3375
Correlation: 0.108547 
 P-Value: 0.130914
Epoch [3/100], Loss: 0.2552
Correlation: 0.030267 
 P-Value: 0.674455
Epoch [4/100], Loss: 0.1765
Correlation: -0.021643 
 P-Value: 0.763930
Epoch [5/100], Loss: 0.1545
Correlation: -0.049026 
 P-Value: 0.496117
Epoch [6/100], Loss: 0.1521
Correlation: -0.000249 
 P-Value: 0.997297
Epoch [7/100], Loss: 0.1380
Correlation: 0.008499 
 P-Value: 0.906125
Epoch [8/100], Loss: 0.1278
Correlation: 0.016180 
 P-Value: 0.822369
Epoch [9/100], Loss: 0.1218
Correlation: -0.031869 
 P-Value: 0.658284
Epoch [10/100], Loss: 0.1142
Correlation: -0.056199 
 P-Value: 0.435190
Epoch [11/100], Loss: 0.1091
Correlation: -0.062883 
 P-Value: 0.382488
Epoch [12/100], Loss: 0.1053
Correlation: -0.059144 
 P-Value: 0.411473
Epoch [13/100], Loss: 0.1029
Correlation: -0.052161 
 P-Value: 0.468948
Epoch [14/100], Loss: 0.1017
Correla

In [470]:
input_size = 361
net_5 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_5.train_net(num_epochs, train_loader_list[5], test_loader_list[5])

Correlation: 0.144569 
 P-Value: 0.043756
Epoch [1/100], Loss: 0.5155
Correlation: 0.138178 
 P-Value: 0.054055
Epoch [2/100], Loss: 0.2062
Correlation: 0.131143 
 P-Value: 0.067634
Epoch [3/100], Loss: 0.1535
Correlation: 0.079983 
 P-Value: 0.266353
Epoch [4/100], Loss: 0.0930
Correlation: 0.061468 
 P-Value: 0.393301
Epoch [5/100], Loss: 0.0769
Correlation: 0.025833 
 P-Value: 0.719987
Epoch [6/100], Loss: 0.0712
Correlation: 0.018111 
 P-Value: 0.801583
Epoch [7/100], Loss: 0.0688
Correlation: 0.076749 
 P-Value: 0.286231
Epoch [8/100], Loss: 0.0621
Correlation: 0.079397 
 P-Value: 0.269879
Epoch [9/100], Loss: 0.0571
Correlation: 0.088932 
 P-Value: 0.216336
Epoch [10/100], Loss: 0.0535
Correlation: 0.060723 
 P-Value: 0.399075
Epoch [11/100], Loss: 0.0492
Correlation: 0.048236 
 P-Value: 0.503087
Epoch [12/100], Loss: 0.0463
Correlation: 0.037398 
 P-Value: 0.603718
Epoch [13/100], Loss: 0.0441
Correlation: 0.024118 
 P-Value: 0.737884
Epoch [14/100], Loss: 0.0422
Correlation: -0

In [471]:
input_size = 369
net_6 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_6.train_net(num_epochs, train_loader_list[6], test_loader_list[6])

Correlation: 0.163928 
 P-Value: 0.022025
Epoch [1/100], Loss: 0.3208
Correlation: 0.171523 
 P-Value: 0.016504
Epoch [2/100], Loss: 0.1148
Correlation: 0.166829 
 P-Value: 0.019752
Epoch [3/100], Loss: 0.0830
Correlation: 0.131358 
 P-Value: 0.067182
Epoch [4/100], Loss: 0.0579
Correlation: 0.107329 
 P-Value: 0.135321
Epoch [5/100], Loss: 0.0473
Correlation: 0.056469 
 P-Value: 0.432977
Epoch [6/100], Loss: 0.0459
Correlation: 0.048758 
 P-Value: 0.498469
Epoch [7/100], Loss: 0.0423
Correlation: 0.083011 
 P-Value: 0.248613
Epoch [8/100], Loss: 0.0383
Correlation: 0.065814 
 P-Value: 0.360652
Epoch [9/100], Loss: 0.0346
Correlation: 0.057067 
 P-Value: 0.428118
Epoch [10/100], Loss: 0.0319
Correlation: 0.013368 
 P-Value: 0.852830
Epoch [11/100], Loss: 0.0289
Correlation: -0.032389 
 P-Value: 0.653073
Epoch [12/100], Loss: 0.0265
Correlation: -0.080106 
 P-Value: 0.265617
Epoch [13/100], Loss: 0.0248
Correlation: -0.089804 
 P-Value: 0.211846
Epoch [14/100], Loss: 0.0237
Correlation:

In [472]:
input_size = 368
net_7 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_7.train_net(num_epochs, train_loader_list[7], test_loader_list[7])

Correlation: 0.209169 
 P-Value: 0.003339
Epoch [1/100], Loss: 0.2187
Correlation: 0.174654 
 P-Value: 0.014605
Epoch [2/100], Loss: 0.1061
Correlation: 0.128870 
 P-Value: 0.072575
Epoch [3/100], Loss: 0.0588
Correlation: -0.078347 
 P-Value: 0.276287
Epoch [4/100], Loss: 0.0421
Correlation: -0.057024 
 P-Value: 0.428461
Epoch [5/100], Loss: 0.0419
Correlation: -0.055446 
 P-Value: 0.441376
Epoch [6/100], Loss: 0.0365
Correlation: 0.001800 
 P-Value: 0.980141
Epoch [7/100], Loss: 0.0312
Correlation: -0.031794 
 P-Value: 0.659036
Epoch [8/100], Loss: 0.0296
Correlation: -0.154756 
 P-Value: 0.030758
Epoch [9/100], Loss: 0.0280
Correlation: -0.193399 
 P-Value: 0.006750
Epoch [10/100], Loss: 0.0273
Correlation: -0.169231 
 P-Value: 0.018026
Epoch [11/100], Loss: 0.0265
Correlation: -0.172395 
 P-Value: 0.015954
Epoch [12/100], Loss: 0.0262
Correlation: -0.193717 
 P-Value: 0.006658
Epoch [13/100], Loss: 0.0258
Correlation: -0.120795 
 P-Value: 0.092543
Epoch [14/100], Loss: 0.0254
Corre